In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import numpy
import xlrd

In [2]:
def re_cate(label):
    if label >2:
        return 1
    else:
        return 0

In [3]:
raw_df = pd.read_csv("data.csv", header = None)
raw_df.rename(columns = {0:'Ae', 1:'Ap', 2:'Rs', 3:'Fz', 4:'label'}, inplace = True)
raw_df['label'] = raw_df.apply(lambda x : re_cate(x.label), axis = 1)
raw_label = raw_df['label'].as_matrix()
raw_df.drop(columns = ['Fz'], axis = 1, inplace = True)
raw_df.drop(columns = ['label'], axis = 1, inplace = True)

F:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [4]:
raw_df['ApxRs'] = raw_df['Ap']*raw_df['Rs']

In [5]:
raw_feature = raw_df.as_matrix()
raw_feature_mean = raw_feature.mean(axis = 0)
raw_feature_std = raw_feature.std(axis = 0)
raw_feature_normalized = (raw_feature - raw_feature_mean)/raw_feature_std

F:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [6]:
raw_feature_normalized

array([[ 1.12115264, -0.50296946, -1.64316767, -0.71075051],
       [ 1.12115264, -0.50296946, -1.46059349, -0.68249005],
       [ 1.12115264, -0.50296946, -1.2780193 , -0.6542296 ],
       ...,
       [-1.56961369,  0.23824869,  1.2780193 ,  0.69096819],
       [-1.56961369,  0.23824869,  1.46059349,  0.75879329],
       [-1.56961369,  0.23824869,  1.64316767,  0.82661839]])

In [7]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [8]:
lr = LogisticRegression(solver='newton-cg', multi_class='multinomial', C=4, tol=1e-6, max_iter=20)

In [9]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    lr.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = lr.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  27 	 NumofIns :  39 	 Pre_Accuracy :  0.6923076923076923 	
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  24 	 NumofIns :  39 	 Pre_Accuracy :  0.6153846153846154 	
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  27 	 NumofIns :  38 	 Pre_Accuracy :  0.7105263157894737 	
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  26 	 NumofIns :  38 	 Pre_Accuracy :  0.6842105263157895 	
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  26 	 NumofIns :  38 	 Pre_Accuracy :  0.6842105263157895 	
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  27 	 NumofIns :  38 	 Pre_Accuracy :  0.7105263157894737 	
#### In Cross Validation 6: ####
NumofIns Precisely Classified :  22 	 NumofIns :  38 	 Pre_Accuracy :  0.5789473684210527 	
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  22 	 NumofIns :  38 	 Pre_Accuracy :  0.5789473684210527 	


In [10]:
SVM = SVC(kernel='rbf',decision_function_shape='ovo',C=20,shrinking =False,tol =1e-6)

In [11]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    SVM.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = SVM.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  30 	 NumofIns :  39 	 Pre_Accuracy :  0.7692307692307693 	
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  35 	 NumofIns :  38 	 Pre_Accuracy :  0.9210526315789473 	
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  27 	 NumofIns :  38 	 Pre_Accuracy :  0.7105263157894737 	
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  34 	 NumofIns :  38 	 Pre_Accuracy :  0.8947368421052632 	
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  28 	 NumofIns :  38 	 Pre_Accuracy :  0.7368421052631579 	
#### In Cross Validation 6: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  31 	 NumofIns :  38 	 Pre_Accuracy :  0.8157894736842105 	


F:\Anaconda\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
F:\Anaconda\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
F:\Anaconda\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
F:\Anaconda\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default 

In [12]:
XGB_C = XGBClassifier(
    #booster = 'gblinear',
    #objective='multi:softmax',
    #num_class=7,#必须要考虑到0的情况。这个数据集里面没有零
    n_estimators=200,
    max_depth=4,
    min_child_weight = 5,
    scale_pos_weight = 5,
    num_boost_round =5,
    max_delta_step=1000,
    alpha =2,
    eta=1
    #colsample_bytree=0.9
    #gamma=5,
    #process_type='update'
)

In [13]:
count_CV = 0
test_acc_record = []
test_pre_record = []

for train_index,test_index in kfold.split(raw_feature_normalized, raw_label):
    XGB_C.fit(raw_feature_normalized[train_index], raw_label[train_index])
    pred_smile_label = XGB_C.predict(raw_feature_normalized[test_index])
    real_label = raw_label[test_index]
    
    test_count_num = 0
    real_label_index = 0
    pre_label_num = 0
    
    for label in pred_smile_label:
        if label == real_label[real_label_index]:
            pre_label_num += 1
        real_label_index += 1
        test_count_num += 1
    
    print('#### In Cross Validation %d: ####'% count_CV)
    count_CV += 1
    print('NumofIns Precisely Classified : ',pre_label_num,'\t',
          'NumofIns : ',test_count_num,'\t',
          'Pre_Accuracy : ',pre_label_num/test_count_num,'\t',)
    
    test_pre_record.append(pre_label_num/test_count_num)

print('mean of NumofIns precisely classified',numpy.mean(test_pre_record))

#### In Cross Validation 0: ####
NumofIns Precisely Classified :  31 	 NumofIns :  39 	 Pre_Accuracy :  0.7948717948717948 	
#### In Cross Validation 1: ####
NumofIns Precisely Classified :  34 	 NumofIns :  39 	 Pre_Accuracy :  0.8717948717948718 	
#### In Cross Validation 2: ####
NumofIns Precisely Classified :  28 	 NumofIns :  38 	 Pre_Accuracy :  0.7368421052631579 	
#### In Cross Validation 3: ####
NumofIns Precisely Classified :  29 	 NumofIns :  38 	 Pre_Accuracy :  0.7631578947368421 	
#### In Cross Validation 4: ####
NumofIns Precisely Classified :  32 	 NumofIns :  38 	 Pre_Accuracy :  0.8421052631578947 	
#### In Cross Validation 5: ####
NumofIns Precisely Classified :  30 	 NumofIns :  38 	 Pre_Accuracy :  0.7894736842105263 	
#### In Cross Validation 6: ####
NumofIns Precisely Classified :  32 	 NumofIns :  38 	 Pre_Accuracy :  0.8421052631578947 	
#### In Cross Validation 7: ####
NumofIns Precisely Classified :  32 	 NumofIns :  38 	 Pre_Accuracy :  0.8421052631578947 	
